In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
print(tf.__version__)

train_dir='E:/data_prediction/Train.csv'
test_dir='E:/data_prediction/Test.csv'

In [ ]:
df_train = pd.read_csv(train_dir)
df_test = pd.read_csv(test_dir)

df_train.drop('id',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)

df_train.head(5)
df_test.head(5)

In [ ]:
CSV_COLUMNS = ['realtionship_status','industry','genre','targeted_sex','average_runtime(minutes_per_week)','airtime','airlocation','ratings','expensive','money_back_guarantee','netgain']
FEATURES_CAT = ['realtionship_status','industry','genre','targeted_sex','airtime','airlocation','expensive','money_back_guarantee']
FEATURES_NUM = ['average_runtime(minutes_per_week)','ratings']
LABEL = CSV_COLUMNS[-1]

df_train['realtionship_status'].unique()
df_train['industry'].unique()
df_train['genre'].unique()
df_train['targeted_sex'].unique()
df_train['airtime'].unique()
df_train['airlocation'].unique()
df_train['money_back_guarantee'].unique()
df_train['expensive'].unique()

In [ ]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )


In [ ]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
def make_feature_cols():
  input_columns1 = tf.feature_column.categorical_column_with_vocabulary_list("realtionship_status",['Widowed','Married-civ-spouse','Divorced','Separated','Never-married','Married-spouse-absent','Married-AF-spouse'],dtype=tf.string)
  input_columns2 = tf.feature_column.categorical_column_with_vocabulary_list("industry",['Auto', 'Pharma', 'Entertainment', 'Political', 'ClassAction','Other'],dtype=tf.string)
  input_columns3 = tf.feature_column.categorical_column_with_vocabulary_list("genre",['Comedy', 'Infomercial', 'Drama', 'Direct', 'Other'],dtype=tf.string)
  input_columns4 = tf.feature_column.categorical_column_with_vocabulary_list("targeted_sex",['Male', 'Female'],dtype=tf.string)
  input_columns5 = tf.feature_column.categorical_column_with_vocabulary_list("airtime",['Primetime', 'Morning', 'Daytime'],dtype=tf.string)
  input_columns6 = tf.feature_column.categorical_column_with_vocabulary_list("airlocation",['United-States', 'Outlying-US(Guam-USVI-etc)', 'Mexico', 'Laos',
       'International', 'India', 'Poland', 'South', 'Italy',
       'Philippines', 'Guatemala', 'Columbia', 'Taiwan', 'England',
       'Canada', 'Germany', 'Thailand', 'Puerto-Rico', 'Nicaragua',
       'Cuba', 'Trinadad&Tobago', 'Yugoslavia', 'El-Salvador', 'Peru',
       'China', 'Japan', 'Vietnam', 'Hong', 'Dominican-Republic',
       'Honduras', 'Hungary', 'Jamaica', 'France', 'Portugal', 'Ireland',
       'Ecuador', 'Iran', 'Scotland', 'Greece', 'Cambodia', 'Haiti',
       'Holand-Netherlands'],dtype=tf.string)
  input_columns7 = tf.feature_column.categorical_column_with_vocabulary_list("money_back_guarantee",['No', 'Yes'],dtype=tf.string)
  input_columns8 = tf.feature_column.categorical_column_with_vocabulary_list("expensive",['High', 'Low', 'Medium'],dtype=tf.string)
  
  final_featcol=[tf.feature_column.numeric_column('average_runtime(minutes_per_week)',dtype=tf.int64),
                 tf.feature_column.numeric_column('ratings',dtype=tf.float64),
                 tf.feature_column.indicator_column(input_columns1),
                 tf.feature_column.indicator_column(input_columns2),
                 tf.feature_column.indicator_column(input_columns3),
                 tf.feature_column.indicator_column(input_columns4),
                 tf.feature_column.indicator_column(input_columns5),
                 tf.feature_column.indicator_column(input_columns6),
                 tf.feature_column.indicator_column(input_columns7),
                 tf.feature_column.indicator_column(input_columns8),

                  ]

  return final_featcol

In [ ]:

model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols())
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10));